#### Scalable table converter 
works ~95% of the time with any table with unlimited rows. Can be expanded to encapsulate more columns. 

In [ ]:
# make sure to have tesseract installed and in your PATH
# pip install pytesseract
import cv2
import pytesseract
import numpy as np
import pandas as pd
import re

#first step, preprocess and crop image
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #Light sharpening the image
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharp = cv2.filter2D(gray, -1, kernel)

    # Gentle threshold
    _, thresh = cv2.threshold(sharp, 160, 255, cv2.THRESH_BINARY_INV)

    #dynamic cropping (you can upload different size tables!!)
    h, w = thresh.shape
    top = int(h * 0.14)
    bottom = int(h * 0.99)
    left = 0
    right = w

    cropped = thresh[top:bottom, left:right]
    cv2.imwrite("final_crop_debug.png", cropped) #for troubleshooting 
    return cropped

#step 2, OCR (optical character recognition)
def extract_text_from_image(image):
    config = r'--oem 3 --psm 6'
    return pytesseract.image_to_string(image, config=config)

#step 3, store raw ocr lines in df
def parse_text_to_dataframe(ocr_text):
    lines = [line.strip() for line in ocr_text.strip().split("\n") if line.strip()]
    return pd.DataFrame({"Raw": lines})

#step 4, convert raw lines to structured columns (assuming it will always be the same 8 columns)
def convert_raw_to_structured(df_raw):
    columns = [
        "Postcode",
        "Number of rental offers",
        "Basic rent in all market segments (€/m²/month)",
        "Basic rent in bottom market segment (€/m²/month)",
        "Basic rent in top market segment (€/m²/month)",
        "Apartment size (m²)",
        "Total housing cost (€/month)",
        "Household purchasing power (€/month)",
        "Extra (if present)"
    ]

    structured_rows = []

    for line in df_raw['Raw']:
        clean = line.replace(",", "").replace("°", "").replace("©", "").replace("=", "")
        clean = re.sub(r"[^0-9A-Za-z/().\s]", "", clean)

        match_label = re.match(r"^(\d{5}|[A-Z][a-zA-Z]+)", clean)
        if not match_label:
            continue

        label = match_label.group(1)
        rest = clean[len(label):].strip()

        #accept n/a and numerical tokens
        tokens = re.findall(r"n/?a|\d+\.\d+\s*\(\d+\)|\d+\s*\(\d+\)|\d+\.\d+|\d+", rest, flags=re.IGNORECASE)

        #default to 8 columns max
        tokens += ["n/a"] * (8 - len(tokens))
        row = [label] + tokens[:8]
        structured_rows.append(row)

    df_structured = pd.DataFrame(structured_rows, columns=columns)
    df_structured = df_structured.fillna("n/a")  # ✅ Fill any remaining blanks

    return df_structured

#step 5, run the whole process!!
def process_table_image(image_path):
    img = preprocess_image(image_path)
    ocr_text = extract_text_from_image(img)

    for i, line in enumerate(ocr_text.strip().split("\n")):
        print(f"Line {i+1}: {line}")

    df_raw = parse_text_to_dataframe(ocr_text)
    df_structured = convert_raw_to_structured(df_raw)
    return df_structured


In [ ]:
#run it here
df = process_table_image("imagine.jpg")
df.to_csv("final_output10.csv", index=False)
print(df)
